In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

2025-07-07 16:54:26.642590: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 16:54:26.649757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751887466.658091   13094 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751887466.660503   13094 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751887466.666892   13094 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [3]:
# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_imdb.h5')
model.summary()

I0000 00:00:1751887467.966989   13094 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1026 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 500, 256)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 500, 256)       │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 128)            │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,740,739 (10.46 MB)

 Trainable params: 2,740,737 (10.46 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
model.get_weights()

[array([[ 6.29046783e-02, -7.40102053e-01,  4.34879392e-01, ...,
         -5.58545232e-01,  4.74583693e-02,  5.14978051e-01],
        [-5.20558991e-02, -8.32785964e-02,  2.93717589e-02, ...,
          2.48580202e-02,  2.14379339e-04,  6.10844977e-02],
        [-2.06722561e-02, -4.85054515e-02,  1.22122586e-01, ...,
         -8.48675147e-02,  3.98941450e-02,  5.75511940e-02],
        ...,
        [ 1.52703792e-01,  2.44462281e-03,  1.73439123e-02, ...,
          5.63957393e-02,  6.16405942e-02, -8.90927240e-02],
        [ 2.97740530e-02,  6.18023127e-02,  5.02433395e-03, ...,
         -4.80964081e-03, -6.11005537e-02,  1.30348448e-02],
        [ 1.31295681e-01,  7.83312246e-02, -1.54340258e-02, ...,
         -8.11702088e-02,  4.51706946e-02, -1.14183977e-01]], dtype=float32),
 array([[ 0.05403183,  0.010295  ,  0.08897924, ...,  0.10088381,
          0.05683538, -0.045393  ],
        [-0.05599511, -0.00015113,  0.07060649, ...,  0.05953872,
          0.05720568,  0.00250059],
        [-

In [5]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [6]:
### Prediction  function

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]



In [7]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.7872692942619324


I0000 00:00:1751887468.558701   25431 service.cc:152] XLA service 0x5fbe1afdd1f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751887468.558713   25431 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 SUPER, Compute Capability 8.9
2025-07-07 16:54:28.568132: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751887468.591111   25431 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1751887468.717086   25431 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
